# Working with LALSuite

Notes on how to add in r-mode resonance features to waveforms with LALSuite.

Some resources:
- LIGO Parameter Estimation paper [here](https://arxiv.org/pdf/1409.7215.pdf). 
- Sivia and Skilling Data Anlaysis book [here](http://pzs.dstu.dp.ua/DataMining/bayes/bibl/Data%20Analysis.pdf). 

# Rossby modes

Rossby modes (r-modes) can be driven by resonances between the orbital and spin frequencies of binary compact objects. This resonance will have some effect on the gravitational wave signal, which we can approximate in the following way. 

At early times, the phase of waveform will be approximately equal to the phase from orbiting point particles: $ \Phi(t) = \Phi_{pp} $. When the frequency of the gravitational wave becomes equal to the resonant frequency, some phase evolution will occur. After the time at which resonance occurs, the phase can be represented as:

$$ \Phi(t) = \Phi_{pp}(t+\Delta t) - \Delta \Phi $$

So the effect of the resonance will cause an overall phase shift and a time shift in the gravitational wave signal. These can be related by:

$$ \Delta \Phi = \frac{\Delta \Phi}{\Delta t} \Delta t = \frac{d\Phi}{dt}\Delta t = \dot{\Phi}_{pp}(t_{0})\Delta t $$

when $\Delta t$ is sufficently small so $ \frac{\Delta \Phi}{\Delta t} $ can be approximated as a derivative. Plugging this into the above equation:

$$ \Phi(t) = \Phi_{pp}(t+\Delta t) - \dot{\Phi}_{pp}(t_{0})\Delta t $$  

Using a taylor expansion: 

$$ \Phi(t) = \Phi_{pp}(t) + \dot{\Phi}_{pp}(t)\Delta t - \dot{\Phi}_{pp}(t_{0})\Delta t $$

The gravitational wave frequency will be twice the orbital frequency, so $ \dot{\Phi}_{pp} = 2\omega $. Using this relationship we can also say $ \Delta t = \frac{\Delta\Phi}{2\omega(t_{0})} $. Substituting these in: 

$$ \Phi(t) = \Phi_{pp}(t) + [\dot{\Phi}_{pp}(t) - \dot{\Phi}_{pp}(t_{0})]\Delta t $$

$$ \Phi(t) = \Phi_{pp}(t) + [2\omega(t) - 2\omega(t_{0})]\Delta t $$

$$ \Phi(t) = \Phi_{pp}(t) + [2\omega(t) - 2\omega(t_{0})]\frac{\Delta\Phi}{2\omega(t_{0})}$$ 

$$ \Phi(t) = \Phi_{pp}(t) + [\frac{\omega(t)}{\omega(t_{0})} - 1]\Delta\Phi$$

In the frequency domain:

$$ \phi(f) = \phi_{pp}(t) + [\frac{f}{f_{0}} - 1]\Delta\phi$$

In a binary system, both neutron stars will undergo resonance. Each will occur at different frequencies, since the resonant frequency is related to the spin frequency of that object. 

$$ \phi(f) = \phi_{pp}(f) + (\frac{f}{f_{01}} - 1)\Delta\phi_{01}+ (\frac{f}{f_{02}} - 1)\Delta\phi_{02}$$

This gives four free parameters: $(f_{01}, \Delta\phi_{01}, f_{02}, \Delta\phi_{02})$

For the purposes of learning how to work with LALSuite, it is easier to assume only one of the objects undergoes resonance, although this is physically unlikely. The equation can thus be simplified as: 

$$ \phi(f) = \phi_{pp}(f) + (\frac{f}{f_{0}} - 1)\Delta\phi $$

$(\frac{f}{f_{0}} - 1)\Delta\phi$ is the correction we will implement into LALSuite. 

# Getting Started

### SSH Keys
See SSH help [here](https://git.ligo.org/help/ssh/README#generating-a-new-ssh-key-pair)

First, get an SSH key on the cluster you are working on. Run:

    ssh-keygen -o -t rsa -b 4096 -C "email@example.com"

In the output, see where you public key has been saved. In this case, it is `/home/yasmeen.asali/.ssh/id_rsa.pub`. Copy this and run:
    
    cat /home/yasmeen.asali/.ssh/id_rsa.pub

Copy the ssh key that prints and paste it into [here](https://git.ligo.org/profile/keys)

### Installing LALSuite
See master lalasuite directory on gitlab [here](https://git.ligo.org/lscsoft/lalsuite/tree/master)

Once you have an ssh key, you have access to gitlab. Next, you want to clone lalsuite and make a branch you can edit. Run:

    git clone git@git.ligo.org:lscsoft/lalsuite.git
    cd lalsuite
    git checkout -b lalsuite-v6.57_branch lalsuite-v6.57
   
If you want lalsuite to install in the working directory, change `DIR_FOR_LALSUITE_TO_INSTALL` to that. Otherwise, change it to wherever you want lalsuite

    ./configure --prefix=DIR_FOR_LALSUITE_TO_INSTALL --enable-swig-python --enable-python
    make
    make install
    
You can change the version of lalsuite you fork from in the `git checkout` line. 

### Using Ctags
See ctags help [here](https://andrew.stwrt.ca/posts/vim-ctags/)

On the cluster, ctags is already installed. To use it, go to the lalsuite directory and run `ctags -R`. This will create a `tags` file in this directory, which vim will look for when you are trying to search classes or methods in ctags. You can set this file as the default tags file in your `.vimrc` by opening it and adding the following line: 

    set tags=~/dev/lalsuite/tags
    
The path to the tags file will change depending on where you have the tags file saved.

Some useful ctags commands:

- mouse over a class or method and run `Crtl ]` to jump to another file with that tag
- `Ctrl t` takes you back a step
- `:tag function_name` goes directly to the tag's definition
- `:ts` or `:tselect` shows the list 
- `:tn` or `:tnext` goes to the next tag in that list
- `:tp` or `:tprev` goes to the previous tag in that list 
- `:tf` or `:tfirst` goes to the first tag of the list 
- `:tl` or `:tlast` goes to the last tag of the list


# Adding something into LALSuite

In order to add some new effect into lalsuite, you need to add it in at two stages. First, you add the effect into lalsimulation where the waveform will be generated with the appropriate phase differences, etc. Then, you need to add something into lalinference. 

Here, we try to add some effect due to r-modes. 

`grep` is a useful tool for finding which files contain certain functions. 

    grep "func" lalinference/src/*.c -l

### LALSimulation

- Start with adding the equation into LALSimIMRPhenomD.c 
    - You will have to declare any variables, don't forget semicolons add the end of the line
    - For example, adding in the RModes phase shift can be done like this:    
```python
REAL8 f0_rmodes = XLALSimInspiralWaveformParamsLookupRModeFrequency(extraParams);
REAL8 delta_phi_rmodes = XLALSimInspiralWaveformParamsLookupRModePhaseDiff(extraParams);
if(freqs->data[i]>f0_rmodes)
{
phi -= t0*(Mf-MfRef) + phi_precalc + (freqs->data[i]/f0_rmodes - 1)*delta_phi_rmodes; /* adding phase offset */
((*htilde)->data->data)[j] = amp0 * amp * cexp(-I * phi);
}
else {
phi -= t0*(Mf-MfRef) + phi_precalc;
((*htilde)->data->data)[j] = amp0 * amp * cexp(-I * phi);
}
```
    - The function names can be anything as long as you are consistent throughout all the files
- Then you have to add the functions from LALSimIMRPhenomD.c to the LALSimInspiralWaveformParams.c file and the header file (change .c to .h) so that the code will know where to look to find this function. In the .c file, add these lines: 
    ```python
    DEFINE_INSERT_FUNC(RModeFrequency, REAL8, "rmode_frequency", 0)
    DEFINE_INSERT_FUNC(RModePhaseDiff, REAL8, "rmode_phase_diff", 0)
    
    DEFINE_LOOKUP_FUNC(RModeFrequency, REAL8, "rmode_frequency", 0)
    DEFINE_LOOKUP_FUNC(RModePhaseDiff, REAL8, "rmode_phase_diff", 0)
    
    DEFINE_ISDEFAULT_FUNC(RModeFrequency, REAL8, "rmode_frequency", 0)
    DEFINE_ISDEFAULT_FUNC(RModePhaseDiff, REAL8, "rmode_phase_diff", 0)
    ```
- Similarly in the header file, add the appropriate lines by checking how it's done for "lambda1"
- Next, you want to compile the code so that it is executeable. You can do this in the lalsuite directory:
    ```bash
    cd ~/dev/lalsuite
    make
    make install
    ```
- After running `make`, you may find errors in your c code (like missing semicolons). If you don't find any errors, then run `make install`
- Now, before going to lalinference to specify how the code should sample over parameters, you want to test if you implemented it correctly. This can be done in python. 

### Checking the waveform in python

Before adding the rest of the pieces into lalinference, you should check that the waveform generating code is correct in lalsimulation. The easiest way to do this is to plot the waveform in python. 

```python
import matplotlib
matplotlib.use("agg")
import matplotlib.pyplot as plt
import numpy as np
import lalsimulation as lalsim
import lal

#first, let's get familiar with lalsim functions

#create an empty dictionary called extraParam where you can save the values for freq and phase
extraParam = lal.CreateDict()
lalsim.SimInspiralWaveformParamsInsertRModeFrequency(extraParam, 128) #setting resonance frequency
lalsim.SimInspiralWaveformParamsInsertRModePhaseDiff(extraParam, 1000) #setting phase shift

#check the values are saved with Lookup functions
lalsim.SimInspiralWaveformParamsLookupRModeFrequency(extraParam) 
lalsim.SimInspiralWaveformParamsLookupRModePhaseDiff(extraParam)

#see what arguements are needed to generate a waveform
lalsim.SimIMRPhenomDGenerateFD?

#now lets generate waveforms with and without phase differences to compare them

#set variables
m1 = 25*lal.MSUN_SI
m2 = 20*lal.MSUN_SI
f0 = 128
del_phi = 1000

#generate waveform with phase difference
extraParam_diff = lal.CreateDict()
lalsim.SimInspiralWaveformParamsInsertRModeFrequency(extraParam_diff, f0)
lalsim.SimInspiralWaveformParamsInsertRModePhaseDiff(extraParam_diff, del_phi)
output_diff = lalsim.SimIMRPhenomDGenerateFD(0., 20., 0.25, m1, 
                                        m2, 0., 0., 20., 1024., 
                                        100e6*lal.PC_SI, extraParam_diff)
#output is the htilde array
htilde_diff = output_diff.data.data

#generate waveform with no phase difference
extraParam_nodiff = lal.CreateDict()
lalsim.SimInspiralWaveformParamsInsertRModeFrequency(extraParam_nodiff, f0)
lalsim.SimInspiralWaveformParamsInsertRModePhaseDiff(extraParam_nodiff, 0)
output_nodiff = lalsim.SimIMRPhenomDGenerateFD(0., 20., 0.25, m1, 
                                        m2, 0., 0., 20., 1024., 
                                        100e6*lal.PC_SI, extraParam_nodiff)
htilde_nodiff = output_nodiff.data.data

#you need to plot it against frequency 
freqs = np.linspace(0, 1024, len(htilde_diff))

plt.figure(1)
plt.plot(freqs, htilde_diff, label="With Phase Difference")
plt.plot(freqs, htilde_nodiff, label="No Phase Difference", linestyle='--')
plt.axvline(f0, label='f0', linestyle='-.', color='k')
plt.legend()
plt.xlim(0, 600)
plt.title("Generated waveforms")
plt.xlabel("Frequency")
plt.ylabel("Strain")
plt.savefig("waveform.pdf")
plt.close(1)
```

You should produce a graph like the following, where you can see that the two waveforms are identical before resonant frequency, then once it hits resonant frequency the phase difference is obvious since we are using the value $\Delta\phi = 1000$. 

![waveform](imgs/waveform.png)

You can also plot the phase shift versus frequency to see how it jumps:

``` python
args_diff = np.angle(htilde_diff)
phi_diff = np.unwrap(args_diff)
args_nodiff = np.angle(htilde_nodiff)
phi_nodiff = np.unwrap(args_nodiff)

plt.figure(2)
plt.plot(freqs, phi_diff, label="With Phase Difference")
plt.plot(freqs, phi_nodiff, label="No Phase Difference")
plt.axvline(f0, label='f0', linestyle='--')
plt.ylim(-500, 1000)
plt.xlim(20, 300)
plt.legend()
plt.title("Phase Shift")
plt.xlabel("Frequency")
plt.ylabel("Cumulative Phase Shift")
plt.savefig("phi.pdf")
plt.close(2)
```

This will produce the following plot:

![phase](imgs/phi.png)

### LALInference

Need to add RMode terms into three files
- lalinference/src/LALInferenceInitCBC.c
- lalinference/src/LALInferenceReadData.c
- lalinference/src/LALInferenceTemplate.c

**LALInferenceInitCBC.c**
- add command line flags in one place
``` c
// RModes
if(LALInferenceGetProcParamVal(commandLine, "--rmodes")){
LALInferenceRegisterUniformVariableREAL8(state, model->params, "RmodeFreq", zero, RmodeFreqMin, RmodeFreqMax, LALINFERENCE_PARAM_LINEAR);
LALInferenceRegisterUniformVariableREAL8(state, model->params, "RmodePhaseDiff", zero, RmodePhaseDiffMin, RmodePhaseDiffMax, LALINFERENCE_PARAM_LINEAR);
}
``` 

**LALInferenceReadData.c**
- add if statements two places
``` c
// RModes
REAL8 rmode_frequency = 0.;
if(LALInferenceGetProcParamVal(commandLine,"--inj-rmode-freq")) {
rmode_frequency = atof(LALInferenceGetProcParamVal(commandLine,"--inj-rmode-freq")->value);
}
REAL8 rmode_phase_diff = 0.;
if(LALInferenceGetProcParamVal(commandLine,"--inj-rmode-phase")) {
rmode_phase_diff = atof(LALInferenceGetProcParamVal(commandLine,"--inj-rmode-phase")->value);
}
if(LALInferenceGetProcParamVal(commandLine,"--inj-rmode-freq")) {
fprintf(stdout,"Injection RMode Frequency set to %f\n", rmode_frequency);
}
if(LALInferenceGetProcParamVal(commandLine,"--inj-rmode-phase")) {
fprintf(stdout,"Injection RMode Phase Diff set to %f\n", rmode_phase_diff);
}
```
- add waveform parameter insert in two places
``` c
XLALSimInspiralWaveformParamsInsertRModeFrequency(LALpars,rmode_frequency);
XLALSimInspiralWaveformParamsInsertRModePhaseDiff(LALpars,rmode_phase_diff);
```

**LALInferenceTemplate.c**
- add same thing in three places
``` c
// RModes
if(LALInferenceCheckVariable(model->params, "RmodeFreq"))
XLALSimInspiralWaveformParamsInsertRModeFrequency(model->LALpars, *(REAL8*) LALInferenceGetVariable(model->params, "RmodeFreq"));
if(LALInferenceCheckVariable(model->params, "RmodePhaseDiff"))
XLALSimInspiralWaveformParamsInsertRModePhaseDiff(model->LALpars, *(REAL8*) LALInferenceGetVariable(model->params, "RmodePhaseDiff"));
```

  


# Creating a virtual environment

PyCBC documentation on creating virtual environments [here](https://pycbc.org/pycbc/latest/html/install.html)



```bash
virtualenv RModeenv/
unset PYTHONPATH
pip 

git format-patch -2 af5e8c6d010c6ca0538442504e3a4e9a882f11e1 #makes two patches for this commit and the one before 

git clone *branch of lalsuite from gitlab*
git log

#undo the last commit while saving changes
git reset HEAD^

```

# Running LALSuite

`lalinference_pipe` will: 
- run lalinference 
- generate posterior samples
- generate html page

```bash
#help and examples
lalinference_pipe --help
vim lalinference/src/lalinference_pipe_example.ini
```

The `.ini` file will specify which engine to use (nested sampling or mcmc), which version of lalsuite, which user, the purpose of the job, etc. You will need a tag to run it using condor, and you can generate a tag [here](https://accounting.ligo.org/user). For my project, the tag is `ligo.dev.o3.cbc.testgr.tiger`. 

# Starting a condor run

1. edit the ini file to make sure its correct 
2. run `lalinference_pipe` 
3. make sure its correct
4. remove `test_rundir` so it doesn't conflict when you run it for real
4. run `lalinference_pipe`  and submit to condor

```bash
vim config/config_template.ini
lalinference_pipe -I injections/bbh_injection.xml -r test_rundir config/config_template.ini
cd test_rundir
vim lalinference_966383880-966384385.sh
#in vim, :set nu! then copy first line, run in terminal
#interrupt the run once it gets to  "Adding cache variables in the sampler"
cd ..
rm -R test_rundir
lalinference_pipe -I injections/bbh_injection.xml -r test_rundir config/config_template.ini --condor-submit
condor_q
watch -n 1 condor_q
```


Homework 
- read sivia and parameter estimation papers
- try to implement the omega / mass relation in lalinference 
    - $ \omega = \frac{4}{3}\frac{M^{2}a}{I} $ where a is dimensionless spin
    - $ I = \frac{8}{3}M^{3}$ where mass is converted to units of time
    - will probably need to add a `--rmodes-constraint` flag into InitCBC.c, and will need to add something into ReadData.c and Template.c 

# Adding rmode constraints to lalinference

How?

    
Solving for $\omega$:    

$$ \omega = \frac{4}{3}\frac{M^{2}a}{I} $$

$$ I = \frac{8}{3}M^{3} $$

$$ \omega = \frac{4}{3}M^{2}a \frac{3}{8M^{3}} $$

$$ \omega = \frac{4}{3}a \frac{3}{8M} $$

$$ \omega = \frac{a}{2M} $$

In the code, `q` and `chirpmass` are recognized variables

```c
if(LALInferenceGetProcParamVal(commandLine, "--rmode-constraint")){
REAL8 mass1 = (pow(q, (2./5.))*(pow((1.0+q),(1./5.))*chirpmass
REAL8 omega = a_spin1/(2*mass1)
}
```

Questions
- how does omega relate with phase diff and freq? 
- should you just define a new parameter for it to sample over? that seems wrong